# Causal Impact



Will Fuks

https://github.com/WillianFuks/tfcausalimpact

[LinkedIn](https://www.linkedin.com/in/willian-fuks-62622217/)


```sh
git clone git@github.com:WillianFuks/pyDataSP-tfcausalimpact.git
cd pyDataSP-tfcausalimpact/
python3.9 -m venv .env
source .env/bin/activate
pip install -r requirements.txt

.env/bin/jupyter notebook
```

In [ ]:
import daft
import os
import collections

os. environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import matplotlib.pyplot as plt
from matplotlib import rc
from IPython.display import HTML
import tensorflow as tf
import tensorflow_probability as tfp
import seaborn as sns
import pandas as pd
import numpy as np

# Attempts to disable TF warnings
tf.get_logger().setLevel('ERROR')
tf.autograph.set_verbosity(tf.compat.v1.logging.ERROR)
import logging
tf.get_logger().setLevel(logging.ERROR)

# TFP namespaces
tfd = tfp.distributions
tfb = tfp.bijectors

# Remove prompt from notebook css
HTML(open('styles/custom.css').read())

## Here's our final destination:

<center><img src="./imgs/tfcausal_plot_original_example.png"/></center>

## Our Journey:

1. Causality

2. Bayesian Time Series

3. Causal Impact
    
    

## Causality Is Simple!

In [ ]:
rc("font", family="serif", size=12)
rc("text", usetex=False)


pgm = daft.PGM(grid_unit=4.0, node_unit=2.5)
rect_params = {"lw": 2}
edge_params = {
    'linewidth': 1,
    'head_width': .8
}

pgm.add_node("rain", r"$Rain$", 0.5, 1.5, scale=1.5, fontsize=24)
pgm.add_node("wet", r"$Wet$", 2.5 + 0.2, 1.5, scale=1.5, fontsize=24)
pgm.add_edge("rain", "wet", plot_params=edge_params)
pgm.render();

## Until It's Not...

In [ ]:
pgm = daft.PGM(grid_unit=4.0, node_unit=4.5)
pgm.add_node("fatigue", r"$Fatigue Train$", 0.5, 1.5, scale=1.5, fontsize=24)
pgm.add_node("perf", r"$Performance$", 2.5 + 0.2, 1.5, scale=1.5, fontsize=24)
pgm.add_edge("fatigue", "perf", plot_params=edge_params)
pgm.render();

In [ ]:
pgm = daft.PGM(grid_unit=4.0, node_unit=2.5)
pgm.add_node("diet", r"$Diet$", 0.5, 3, scale=1.5, fontsize=24)
pgm.add_node("rest", r"$Rest$", 0.5, 1.5, scale=1.5, fontsize=24)
pgm.add_node("volume", r"$Volume$", 0.5, 0, scale=1.5, fontsize=24)
pgm.add_node("fatigue", r"$Fatigue$", 0.5, -1.5, scale=1.5, fontsize=24)
pgm.add_node("perf", r"$Performance$", 2.5 + 0.2, 1.5, scale=2.5, fontsize=24)
pgm.add_edge("diet", "perf", plot_params=edge_params)
pgm.add_edge("volume", "perf", plot_params=edge_params)
pgm.add_edge("rest", "perf", plot_params=edge_params)
pgm.add_edge("fatigue", "perf", plot_params=edge_params)


# pgm.add_edge("diet", "rest", plot_params=edge_params)
# pgm.add_edge("rest", "diet", plot_params=edge_params)
# pgm.add_edge("diet", "volume", plot_params=edge_params)
# pgm.add_edge("volume", "diet", plot_params=edge_params)
# pgm.add_edge("diet", "fatigue", plot_params=edge_params)
# pgm.add_edge("fatigue", "diet", plot_params=edge_params)

pgm.render();

## So How To Compute Causality?!


### Correlations?

## Let's Explore The Idea

### Tensorflow Probability

(Random Variables)

In [ ]:
import os
import tensorflow as tf
import tensorflow_probability as tfp


# tf.get_logger().setLevel('INFO')
# os. environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
# tf.autograph.set_verbosity(1)

tfd = tfp.distributions
tfb = tfp.bijectors

In [ ]:
X = tfd.Normal(loc=2, scale=1)
sns.displot(X.sample(1000), kde=True);

In [ ]:
b_X = tfd.TransformedDistribution(
    tfd.Normal(loc=2, scale=1),
    bijector=tfb.Exp()
)
sns.displot(b_X.sample(1000), kde=True);

## Suppose This (Simplified) Structure

In [ ]:
pgm = daft.PGM(grid_unit=4.0, node_unit=2.5)
pgm.add_node("diet", r"$Diet$", 0., 0., scale=1.5, fontsize=24)
pgm.add_node("volume", r"$Volume$", 1.25, 0, scale=1.5, fontsize=24)
pgm.add_node("perf", r"$Performance$", 3., 0., scale=2.5, fontsize=24)
pgm.add_edge("diet", "volume", plot_params=edge_params)
pgm.add_edge("volume", "perf", plot_params=edge_params)

pgm.render();

## And Respective Data

In [ ]:
dist = tfd.JointDistributionNamed(
    {
        'diet': tfd.Normal(loc=3, scale=1),
        'volume': lambda diet: tfd.Normal(diet * 2, scale=0.5),
        'performance': lambda volume: tfd.Normal(volume * 1.3, scale=0.3)
    }
)
data = dist.sample(3000)
data = pd.DataFrame(data, columns=['performance', 'diet', 'volume'])
# data.set_index(pd.date_range(start='20200101', periods=len(data)), inplace=True)
# data

## Linear Regression Keras

In [ ]:
import tensorflow as tf


linear_model = tf.keras.Sequential([
    tf.keras.layers.Dense(units=1, use_bias=False)
])

linear_model.compile(
    optimizer=tf.optimizers.Adam(learning_rate=0.1),
    loss=tf.keras.losses.MeanSquaredError()
)

linear_model.fit(
    data[['diet']],
    data['performance'],
    epochs=100,
    verbose=0,
)

w = linear_model.get_weights()
print(f"Linear Relationship is: {w[0][0][0]:.2f}")

In [ ]:
linear_model = tf.keras.Sequential([
    tf.keras.layers.Dense(units=1, use_bias=False)
])

linear_model.compile(
    optimizer=tf.optimizers.Adam(learning_rate=0.1),
    loss=tf.keras.losses.MeanSquaredError()
)

linear_model.fit(
    data[['volume']],
    data['performance'],
    epochs=100,
    verbose=0,
)

w = linear_model.get_weights()
print(f"Linear Relationship is: {w[0][0][0]:.2f}")

In [ ]:
linear_model = tf.keras.Sequential([
    tf.keras.layers.Dense(units=1, use_bias=False)
])

linear_model.compile(
    optimizer=tf.optimizers.Adam(learning_rate=0.1),
    loss=tf.keras.losses.MeanSquaredError()
)

linear_model.fit(
    data[['diet', 'volume']],
    data['performance'],
    epochs=100,
    verbose=0,
)

w = linear_model.get_weights()
print(f"Linear Relationship is: {w[0]}")

## Bayesian Linear Regression

### Recipe!

$$\begin{equation} \label{eq1}
\begin{split}
P(A|B) & = \frac{P(B|A)P(A)}{P(B)} = \frac{P(A, B)}{P(B)}
\end{split}
\end{equation}
$$

$$\begin{equation} \label{eq1}
\begin{split}
P(\theta|D) & = \frac{P(\theta, D)}{P(D)}
\end{split}
\end{equation}
$$

## Step 1: Priors

In [ ]:
pgm = daft.PGM(grid_unit=4.0, node_unit=2.5)
pgm.add_node("diet", r"$w_{diet}$", 0., 0., scale=1.5, fontsize=24)
pgm.add_node("volume", r"$w_{volume}$", 1.25, 0, scale=1.5, fontsize=24)
pgm.add_node("sigma", r"$\sigma^2$", 2.5, 0, scale=1.5, fontsize=24)
pgm.add_node("perf", r"$Performance$", 1.25, -2, scale=2.25, fontsize=24)

pgm.add_edge("diet", "perf", plot_params=edge_params)
pgm.add_edge("volume", "perf", plot_params=edge_params)
pgm.add_edge("sigma", "perf", plot_params=edge_params)
pgm.render();

\begin{equation} \label{eq1}
\begin{split}
w_{diet} & \sim N(3, 5)   \\
w_{volume} & \sim N(3, 5) \\
\sigma^2 & \sim Exp(2) \\
performance & \sim N(w_{diet} \cdot x_{diet} + w_{volume} \cdot x_{volume}, \sigma^2)
\end{split}
\end{equation}

In [ ]:
joint_dist = tfd.JointDistributionNamedAutoBatched(dict(
    sigma=tfd.Exponential(2),
    
    w_diet=tfd.Normal(loc=3, scale=5),
    
    w_volume=tfd.Normal(loc=3, scale=5),
    
    performance=lambda sigma, w_diet, w_volume: tfd.Normal(loc=data['diet'].values * w_diet + data['volume'].values * w_volume, scale=sigma)

))

In [ ]:
prior_samples = joint_dist.sample(100)
nrows = 3
labels = ['sigma', 'w_diet', 'w_volume']
fig, axes = plt.subplots(nrows=nrows, ncols=1, figsize=(10, 8))

for i in range(nrows):
    sns.histplot(prior_samples[labels[i]], kde=True, ax=axes[i], label=labels[i]);
    axes[i].legend();

# Step 2: Joint Distribution

$$P(\theta, X)$$

In [ ]:
def target_log_prob_fn(sigma, w_diet, w_volume):
    return joint_dist.log_prob(sigma=sigma, w_diet=w_diet, w_volume=w_volume, performance=data['performance'].values)

# Step 3: MCMC

In [ ]:
num_results = int(1e4)
num_burnin_steps = int(1e3)


kernel = tfp.mcmc.HamiltonianMonteCarlo(
    target_log_prob_fn=target_log_prob_fn,
    step_size=0.3,
    num_leapfrog_steps=3
)

kernel = tfp.mcmc.TransformedTransitionKernel(
    inner_kernel=kernel,
    bijector=[tfb.Exp(), tfb.Identity(), tfb.Identity()]
)

kernel = tfp.mcmc.DualAveragingStepSizeAdaptation(
    inner_kernel=kernel,
    num_adaptation_steps=int(num_burnin_steps * 0.8)
)

In [ ]:
@tf.function(autograph=False)
def sample_chain():
    return tfp.mcmc.sample_chain(
        num_results=num_results,
        num_burnin_steps=num_burnin_steps,
        kernel=kernel,
        current_state=[
            tf.constant(0.5, dtype=tf.float32),
            tf.constant(0.3, dtype=tf.float32),
            tf.constant(0.2, dtype=tf.float32)
        ],
        trace_fn=lambda _, pkr: [pkr.inner_results.inner_results.accepted_results.step_size,
                                 pkr.inner_results.inner_results.log_accept_ratio]
    )

In [ ]:
samples, [step_size, log_accept_ratio] = sample_chain()

In [ ]:
p_accept = tf.reduce_mean(tf.exp(tf.minimum(log_accept_ratio, 0.)))
p_accept

In [ ]:
nrows = 3
labels = ['$\sigma$', 'w_diet', 'w_volume']
fig, axes = plt.subplots(nrows=nrows, ncols=1, figsize=(10, 8))

for i in range(nrows):
    sns.histplot(samples[i], kde=True, ax=axes[i], label=labels[i]);
    axes[i].legend();

In [ ]:
sigmas, w_diets, w_volumes = samples

In [ ]:
performance_estimated = (
    tf.linalg.matmul(data['diet'].values[..., tf.newaxis], w_diets[..., tf.newaxis], transpose_b=True) +
    tf.linalg.matmul(data['volume'].values[..., tf.newaxis], w_volumes[..., tf.newaxis], transpose_b=True)
)

In [ ]:
quanties_performance = tf.transpose(tfp.stats.percentile(
    performance_estimated, [2.5, 97.5], axis=1, interpolation=None, keepdims=False,
))

In [ ]:
mean_y = tf.math.reduce_mean(performance_estimated, axis=1)
mean_y

In [ ]:
std_y = tf.math.reduce_std(performance_estimated, axis=1)
std_y

In [ ]:
fig, ax = plt.subplots(figsize=(9, 8)) 
ax.errorbar(
    x=data['performance'], 
    y=mean_y, 
    yerr=2*std_y,
    fmt='o',
    capsize=2,
    label='predictions +/- CI'
)

sns.regplot(
    x=data['performance'], 
    y=mean_y, 
    scatter=False,
    line_kws=dict(alpha=0.5), 
    label='performance / predicted performance', 
    truncate=False,
    ax=ax
);
ax.set(ylabel='predicted performance');
plt.legend();

When we fit `diet` and `volume` together

`diet` seems to lose causality!!

## Interesting Problem

In [ ]:
pgm = daft.PGM(grid_unit=4.0, node_unit=2.5)
pgm.add_node("diet", r"$Diet$", 0.5, 3, scale=1.5, fontsize=24)
pgm.add_node("rest", r"$Rest$", 0.5, 1.5, scale=1.5, fontsize=24)
pgm.add_node("volume", r"$Volume$", 0.5, 0, scale=1.5, fontsize=24)
pgm.add_node("fatigue", r"$Fatigue$", 0.5, -1.5, scale=1.5, fontsize=24)
pgm.add_node("question", r"$?$", 2.5 + 0.2, 1.5, scale=1.5, fontsize=24)
pgm.add_node("perf", r"$Performance$", 5, 1.5, scale=2.5, fontsize=24)

pgm.add_edge("diet", "question", plot_params=edge_params)
pgm.add_edge("volume", "question", plot_params=edge_params)
pgm.add_edge("rest", "question", plot_params=edge_params)
pgm.add_edge("fatigue", "question", plot_params=edge_params)
pgm.add_edge("question", "perf", plot_params=edge_params)


pgm.render();

## Correlations won't work

## Better Solution?


## A/B Test!

<center><img src="imgs/ab.png" /></center>

## Not so fast...

 <center><img src='imgs/Store-WP.png'/></center>

## Control Group Fail...

## Solution: Quasi Experiments

<center><img src="imgs/ci.png"/></center>

## Structural Time Series

<center><img src="imgs/stsequation.png" /></center>

<center><img src="imgs/stsgraph.png" /></center>

## Important thing is: Structures

- [AutoRegressive](https://github.com/tensorflow/probability/blob/v0.11.1/tensorflow_probability/python/sts/autoregressive.py#L258)
- [DynamicRegression](https://github.com/tensorflow/probability/blob/v0.11.1/tensorflow_probability/python/sts/dynamic_regression.py#L230)
- [LocalLevel](https://github.com/tensorflow/probability/blob/v0.11.1/tensorflow_probability/python/sts/local_level.py#L254)
- [Seasonal](https://github.com/tensorflow/probability/blob/v0.11.1/tensorflow_probability/python/sts/seasonal.py#L688)
- [LocalLinearTrend](https://github.com/tensorflow/probability/blob/v0.11.1/tensorflow_probability/python/sts/local_linear_trend.py#L222)
- [SemiLocalLinearTrend](https://github.com/tensorflow/probability/blob/v0.11.1/tensorflow_probability/python/sts/semilocal_linear_trend.py#L294)
- [SmoothSeasonal](https://github.com/tensorflow/probability/blob/v0.11.1/tensorflow_probability/python/sts/smooth_seasonal.py#L321)
- [Regression](https://github.com/tensorflow/probability/blob/v0.11.1/tensorflow_probability/python/sts/regression.py#L51)
- [SparseLinearRegression](https://github.com/tensorflow/probability/blob/v0.11.1/tensorflow_probability/python/sts/regression.py#L264)

## Local Level

$$\mu_t = \mu_{t-1} + Normal(0, \sigma^2_{\mu})$$

In [ ]:
local_level_model = tfp.sts.LocalLevelStateSpaceModel(
    num_timesteps=20,
    level_scale=.1,
    initial_state_prior=tfd.MultivariateNormalDiag(scale_diag=[1.])
)

s = local_level_model.sample(1)
plt.plot(tf.squeeze(s));

In [ ]:
local_level_model.log_prob(s)

## Local And Regression (Model Fit)

In [ ]:
data = pd.read_csv('https://raw.githubusercontent.com/WillianFuks/tfcausalimpact/master/tests/fixtures/arma_data.csv', dtype=np.float32)[['y', 'X']]
data.iloc[70:, 0] += 5
data.plot()
plt.axvline(70, 0, 130, linestyle='--', color='r', linewidth=0.85);

y = tf.cast(data['y'].values[:70], tf.float32)

In [ ]:
local_level = tfp.sts.LocalLevel(
    observed_time_series=y
)

In [ ]:
regression = tfp.sts.LinearRegression(
    design_matrix=tf.cast(data['X'].values[..., tf.newaxis], tf.float32)
)

In [ ]:
model = tfp.sts.Sum([local_level, regression], observed_time_series=y)

In [ ]:
samples, _ = tfp.sts.fit_with_hmc(model, y)

In [ ]:
one_step_predictive_dist = tfp.sts.one_step_predictive(model, observed_time_series=y, parameter_samples=samples)

In [ ]:
predictive_means = one_step_predictive_dist.mean()
predictive_means

In [ ]:
predictive_scales = one_step_predictive_dist.stddev()
predictive_scales

In [ ]:
plt.figure(figsize=(10, 9))
color = (1.0, 0.4981, 0.0549)
plt.plot(y, label='y', color='k')
# plt.plot(predictive_means[1:], color=color, label='predictive mean')
plt.fill_between(
    np.arange(1, 70),
    predictive_means[1:70] - predictive_scales[1:70],
    predictive_means[1:70] + predictive_scales[1:70],
    alpha=0.4,
    color=color,
    label='predictive std'
)
plt.legend();

In [ ]:
forecast_dist = tfp.sts.forecast(model, observed_time_series=y, parameter_samples=samples, num_steps_forecast=30)

In [ ]:
forecast_means = tf.squeeze(forecast_dist.mean())
forecast_scales = tf.squeeze(forecast_dist.stddev())

In [ ]:
plt.figure(figsize=(10, 9))
plt.plot(data['y'], label='y', color='k')
plt.fill_between(
    np.arange(1, 70),
    predictive_means[1:] - predictive_scales[1:],
    predictive_means[1:] + predictive_scales[1:],
    alpha=0.4,
    color=color,
    label='predictive std'
)
plt.plot(np.arange(70, 100), forecast_means, color='r', label='mean forecast')
plt.fill_between(
    np.arange(70, 100),
    forecast_means - forecast_scales,
    forecast_means + forecast_scales,
    alpha=0.4,
    color='red',
    label='forecast std'
)
plt.legend();

## How to obtain causal impact?

### tfcausalimpact

In [ ]:
!pip install tfcausalimpact > /dev/null

In [ ]:
from causalimpact import CausalImpact


data = pd.read_csv('https://raw.githubusercontent.com/WillianFuks/tfcausalimpact/master/tests/fixtures/arma_data.csv')[['y', 'X']]
data.iloc[70:, 0] += 5

pre_period = [0, 69]
post_period = [70, 99]

ci = CausalImpact(data, pre_period, post_period)

In [ ]:
print(ci.summary())

In [ ]:
print(ci.summary(output='report'))

In [ ]:
ci.plot(figsize=(15, 15))

In [ ]:
ci.model.components_by_name

In [ ]:
ci.model_samples

In [ ]:
# https://www.tensorflow.org/probability/examples/Structural_Time_Series_Modeling_Case_Studies_Atmospheric_CO2_and_Electricity_Demand

component_dists = tfp.sts.decompose_by_component(
    ci.model,
    observed_time_series=y,
    parameter_samples=ci.model_samples
)

component_means, component_stddevs = (
    {k.name: c.mean() for k, c in component_dists.items()},
    {k.name: c.stddev() for k, c in component_dists.items()}
)

def plot_components(dates,
                    component_means_dict,
                    component_stddevs_dict):
  x_locator, x_formatter = None, None
  colors = sns.color_palette()
  c1, c2 = colors[0], colors[1]

  axes_dict = collections.OrderedDict()
  num_components = len(component_means_dict)
  fig = plt.figure(figsize=(12, 2.5 * num_components))
  for i, component_name in enumerate(component_means_dict.keys()):
    component_mean = component_means_dict[component_name]
    component_stddev = component_stddevs_dict[component_name]

    ax = fig.add_subplot(num_components, 1, 1 + i)
    ax.plot(dates, component_mean, lw=2)
    ax.fill_between(dates, component_mean-2*component_stddev,
                    component_mean+2*component_stddev,
                    color=c2, alpha=0.5)
    ax.set_title(component_name)
    if x_locator is not None:
      ax.xaxis.set_major_locator(x_locator)
      ax.xaxis.set_major_formatter(x_formatter)
    axes_dict[component_name] = ax
#   fig.autofmt_xdate()
  fig.tight_layout()


plot_components(np.arange(0, 70), component_means, component_stddevs);

## Real Example: Bitcoin

In [ ]:
! pip install pandas-datareader > /dev/null

In [ ]:
import datetime
import pandas_datareader as pdr


btc_data = pdr.get_data_yahoo(['BTC-USD'], 
                              start=datetime.datetime(2018, 1, 1), 
                              end=datetime.datetime(2020, 12, 3))['Close']
btc_data = btc_data.reset_index().drop_duplicates(subset='Date', keep='last').set_index('Date').sort_index()
btc_data = btc_data.resample('D').fillna('nearest')
X_data = pdr.get_data_yahoo(['TWTR', 'GOOGL', 'AAPL', 'MSFT', 'AMZN', 'FB', 'GOLD'], 
                            start=datetime.datetime(2018, 1, 1), 
                            end=datetime.datetime(2020, 12, 2))['Close']
X_data = X_data.reset_index().drop_duplicates(subset='Date', keep='last').set_index('Date').sort_index()
X_data = X_data.resample('D').fillna('nearest')
data = pd.concat([btc_data, X_data], axis=1)
data.dropna(inplace=True)
data = data.resample('W-Wed').last()  # Weekly is easier to process. We select Wednesday so 2020-10-21 is available.
data = data.astype(np.float32)

np.log(data).plot(figsize=(15, 12))
plt.axvline('2020-10-14', 0, np.max(data['BTC-USD']), lw=2, ls='--', c='red', label='PayPal Impact')
plt.legend(loc='upper left');

In [ ]:
pre_period=['20180103', '20201014']
post_period=['20201021', '20201125']
ci = CausalImpact(data, pre_period, post_period, model_args={'fit_method': 'vi'})

In [ ]:
print(ci.summary())

In [ ]:
ci.plot(figsize=(15, 15))

## Tips

## Q - How select covariates?

## A - Yes!

tfp.sts.SparseLinearRegression

## Decompose by [`statsmodels`](https://github.com/statsmodels/statsmodels)

In [ ]:
!pip install statsmodels > /dev/null

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose


fig, axes = plt.subplots(4, 1, figsize=(15, 15))
fig.tight_layout()

res = seasonal_decompose(data['BTC-USD'])

axes[0].plot(res.observed)
axes[0].set_title('Observed')

axes[1].plot(res.trend)
axes[1].set_title('Trend')

axes[2].plot(res.seasonal)
axes[2].set_title('Seasonal')

axes[3].plot(res.resid)
axes[3].set_title('Residuals');



# Fourier FTW

In [ ]:
# https://colab.research.google.com/drive/10VADEg8F5t_FuryEf_ObFfeIFwX-CxII?usp=sharing#scrollTo=UyA6K6GTyJqF

from numpy.fft import rfft, irfft, rfftfreq


def annot_max(x, y, ax=None):
    xmax = x[np.argmax(y)]
    ymax = y.max()
    text= "x={:.3f}, y={:.3f}".format(xmax, ymax) 
    #text = f"{xmax=}, {ymax=}, (period: {1./xmax} days)" #Eh, Colab has Python 3.6 ... 
    text = f"x={xmax:.3f}, y={ymax:.3f}, (period: {(1./xmax):.2f} weeks)"
    if not ax:
        ax=plt.gca()
    bbox_props = dict(boxstyle="square,pad=0.3", fc="w", ec="k", lw=0.72)
    arrowprops=dict(arrowstyle="->",connectionstyle="angle,angleA=0,angleB=60")
    kw = dict(xycoords='data',textcoords="axes fraction",
              arrowprops=arrowprops, bbox=bbox_props, ha="right", va="top")
    ax.annotate(text, xy=(xmax, ymax),  xytext=(0.94, 0.96), **kw)

    
y = data['BTC-USD']
nobs = len(y)
btc_ft = np.abs(rfft(y))
btc_freq = rfftfreq(nobs)
plt.plot(btc_freq[2:], btc_ft[2:])
annot_max(btc_freq[2:], btc_ft[2: ]);

# Cross Validation

In [ ]:
plt.figure(figsize=(15, 10))
plt.plot(y)
plt.axvline(pd.to_datetime('2018-09-01'), 0, 19000, c='r')
plt.axvline(pd.to_datetime('2019-09-01'), 0, 19000, c='g')

plt.text(pd.to_datetime('2018-03-01'), 18000, 'train', bbox=dict(fill=False, edgecolor='k', linewidth=0.5), fontdict={'fontsize': 20})
plt.text(pd.to_datetime('2019-01-01'), 18000, 'cross-validate', bbox=dict(fill=False, edgecolor='k', linewidth=0.5), fontdict={'fontsize': 20})
plt.text(pd.to_datetime('2020-02-01'), 18000, 'causal impact', bbox=dict(fill=False, edgecolor='k', linewidth=0.5), fontdict={'fontsize': 20});

## And that's pretty much it ;)!

## Thanks!